In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
import tensorflow as tf

try:
    tf_version = tf.__version__
    print("TensorFlow version:", tf_version)
    print("TensorFlow is installed.")
except ImportError:
    print("TensorFlow is not installed.")

TensorFlow version: 2.15.0
TensorFlow is installed.


In [6]:
import os
import cv2
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os
import numpy as np
import tensorflow as tf
import cv2
from sklearn.preprocessing import OneHotEncoder
import csv
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import datetime  # Added for TensorBoard log directory

In [37]:
# Specify the directory where your CSV files are located
directory_path = 'C:/yash/Projects/cappstone/Cappstone-theft/New folder/'
videos_directory = 'C:/yash/Projects/cappstone/Cappstone-theft/shop'

def prepareParentVideoPath(video_name):
    video_path = ""
    done = False
    for c in video_name:
        if(done and c == '_'):
            return video_path
        video_path += c
        if(c == '_'):
            done = True
    return ""
        
# Initialize empty lists to store video names and abnormal behaviors
video_paths = []
labels = []
counter = 0
# Iterate over each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory_path, filename)
        print(file_path)
        # Open the CSV file and read its contents
        with open(file_path, 'r') as csv_file:
            csv_reader = csv.reader(csv_file)

            # Skip the header if it exists
            next(csv_reader, None)

            # Iterate over each row in the CSV file
            for row in csv_reader:
                # Assuming the first column is the video name and the second column is the abnormal behavior
                video_path = videos_directory + '/' + filename[:-4] + '/' + prepareParentVideoPath(row[0]) + '.mp4' + '/' + row[0] + '.mp4' #row[0]
                label = int(row[2]) # Convert the value to an integer

                # Append the values to the respective lists
                video_paths.append(video_path)
                labels.append(label)
                counter +=1
                if (counter == 100):
                    break
            if (counter == 100):
                break
        if (counter == 100):
            break
           

print(video_paths)
print(labels)
#print(abnormal_behaviors)

C:/yash/Projects/cappstone/Cappstone-theft/New folder/Shoplifting.csv
['C:/yash/Projects/cappstone/Cappstone-theft/shop/Shoplifting/Shoplifting001_x264.mp4/Shoplifting001_x264_1.mp4', 'C:/yash/Projects/cappstone/Cappstone-theft/shop/Shoplifting/Shoplifting001_x264.mp4/Shoplifting001_x264_2.mp4', 'C:/yash/Projects/cappstone/Cappstone-theft/shop/Shoplifting/Shoplifting001_x264.mp4/Shoplifting001_x264_3.mp4', 'C:/yash/Projects/cappstone/Cappstone-theft/shop/Shoplifting/Shoplifting001_x264.mp4/Shoplifting001_x264_4.mp4', 'C:/yash/Projects/cappstone/Cappstone-theft/shop/Shoplifting/Shoplifting001_x264.mp4/Shoplifting001_x264_5.mp4', 'C:/yash/Projects/cappstone/Cappstone-theft/shop/Shoplifting/Shoplifting001_x264.mp4/Shoplifting001_x264_6.mp4', 'C:/yash/Projects/cappstone/Cappstone-theft/shop/Shoplifting/Shoplifting001_x264.mp4/Shoplifting001_x264_7.mp4', 'C:/yash/Projects/cappstone/Cappstone-theft/shop/Shoplifting/Shoplifting001_x264.mp4/Shoplifting001_x264_8.mp4', 'C:/yash/Projects/cappsto

In [38]:
# Function to extract frames from a video file
def extract_frames(video_path, num_frames=30, frame_size=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while len(frames) < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, frame_size)
        frames.append(frame)

    cap.release()
    return np.array(frames)

# Generator to load and preprocess frames on-the-fly
def frame_generator(video_paths, labels, batch_size=32, num_frames=30):
    num_videos = len(video_paths)
    while True:
        for start in range(0, num_videos, batch_size):
            frames_batch = []
            labels_batch = labels[start:start + batch_size]

            for video_path in video_paths[start:start + batch_size]:
                frames = extract_frames(video_path, num_frames=num_frames)
                frames_batch.append(frames)

            frames_batch = np.array(frames_batch)
            frames_batch = preprocess_input(frames_batch)

            yield frames_batch, np.array(labels_batch)

# train_video_paths = video_paths
# train_labels = labels
# # Split the data into training and validation sets
train_video_paths, val_video_paths, train_labels, val_labels = train_test_split(
    video_paths, labels, test_size=0.2, random_state=42
)

In [34]:
# batch_size = 32
# train_generator = frame_generator(train_video_paths, train_labels, batch_size=batch_size)
# validation_generator = frame_generator(val_video_paths, val_labels, batch_size=batch_size)


In [35]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
num_frames=30
height=224
width=224
channels=3
# Set the input shape based on your video frame dimensions
input_shape = (num_frames, height, width, channels)

# Define the 3D CNN model
model = Sequential()

model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Two output classes: shoplifting_yes and shoplifting_no

In [36]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Create an ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# train_generator = frame_generator(train_video_paths, train_labels, batch_size=batch_size)
# validation_generator = frame_generator(val_video_paths, val_labels, batch_size=batch_size)

# Load and preprocess the video dataset using the generator
train_generators=[]
for path in train_video_paths:
    
    train_generator = datagen.flow_from_directory(
        path,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    train_generators.append(train_generator)
cnt_train_generator = train_generators[0]
for i in range(1,len(train_generators)):
    cnt_train_generator = cnt_train_generator.concatenate(train_generators[i])

NotADirectoryError: [WinError 267] The directory name is invalid: 'C:/yash/Projects/cappstone/Cappstone-theft/shop/Shoplifting/Shoplifting001_x264.mp4/Shoplifting001_x264_1.mp4'

In [36]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Create an ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# train_generator = frame_generator(train_video_paths, train_labels, batch_size=batch_size)
# validation_generator = frame_generator(val_video_paths, val_labels, batch_size=batch_size)

# Load and preprocess the video dataset using the generator
train_generators=[]
for path in train_video_paths:
    
    train_generator = datagen.flow_from_directory(
        path,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    train_generators.append(train_generator)
cnt_train_generator = train_generators[0]
for i in range(1,len(train_generators)):
    cnt_train_generator = cnt_train_generator.concatenate(train_generators[i])

NotADirectoryError: [WinError 267] The directory name is invalid: 'C:/yash/Projects/cappstone/Cappstone-theft/shop/Shoplifting/Shoplifting001_x264.mp4/Shoplifting001_x264_1.mp4'

In [ ]:
validation_generator = datagen.flow_from_directory(
    val_video_paths,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Train the model
epochs = 10
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Save the trained model
model.save('video_classification_model.h5')